# Sample Notebook

## install dependencies

In [12]:
%pip install transformers[torch]
%pip install beautifulsoup4
%pip install boto3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Define helper methods

In [13]:
import os

def extract_text_from_s3_file(bucket_name, file_name, output_filename):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=file_name)
    
    # Use a streaming approach to process the file line by line
    bytestream = io.BytesIO(obj['Body'].read())
    
    with open(output_filename, 'w') as outfile:
        for line in bytestream:
            item = json.loads(line.decode('utf-8'))
            raw_html = item.get('RawHtml', '')
            text = extract_text_from_html(raw_html)
            outfile.write(text + "\n")
    
    print(f"Processing complete. Text written to: {output_filename}")
    
    # Upload the output file to S3
    s3.upload_file(output_filename, bucket_name, output_filename)
    print(f"File uploaded to: {bucket_name}/{output_filename}")
    
    # Delete the local output file
    os.remove(output_filename)
    print(f"Local file deleted: {output_filename}")


ModuleNotFoundError: No module named 'boto3'

## Prepare Data

### extract text from blogs json and upload to S3

In [ ]:
bucket_name = "awsc.datascience.objecjstore"
file_name = "raw_data/allBlogList20230613Detailed.json"

text_list = extract_text_from_s3_file(bucket_name, file_name)

## Load Model

In [9]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'bert-base-uncased'

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Run Vector

In [15]:
# Define a sample text
text = "Hello, this is a test."

# Encode the text to get the input tensors
inputs = tokenizer(text, return_tensors='pt')

# Run the text through the model to get the embeddings
outputs = model(**inputs)

# Use the average of the last hidden state as the text's embedding
embeddings = outputs.last_hidden_state.mean(dim=1)

# Convert the tensor to a numpy array
vectors = embeddings.detach().numpy()

print (vectors[:, :50])

[[-0.77382195  0.10704014  0.44775397 -0.08637124  0.35413992  0.03406591
   0.16264133  0.58041024 -0.23546062 -0.14814864 -0.0935005  -0.39231786
  -0.33704147  0.2717751  -0.12507012  0.35159644  0.41959     0.1162833
  -0.18021566  0.37518147 -0.16880287  0.17903076 -0.23813213  0.40838736
   0.16964942 -0.0612941   0.16057596  0.03659857 -0.309846   -0.03385863
   0.35843623  0.06824324  0.19181494 -0.22375077 -0.21855049 -0.2087363
   0.5018591   0.10408244 -0.30850494 -0.00264207 -0.65153766 -0.41132504
   0.11561029  0.09209279 -0.3221398  -0.2758163   0.3976515  -0.13493092
   0.15201087  0.41245016]]
